In [72]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from collections import Counter
from sklearn.metrics import accuracy_score, log_loss

import pickle
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier, VotingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold, learning_curve

sns.set(style='white', context='notebook', palette='deep')
import lightgbm
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# read pickle
with open('./data/X.pickle', 'rb') as handle:
    X = pickle.load(handle)
    
with open('./data/y.pickle', 'rb') as handle:
    y = pickle.load(handle)

In [83]:
max_depth_tree =5
num_trees=100

m1 = lightgbm.LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        importance_type='split', learning_rate=1, max_depth=max_depth_tree,
        min_child_samples=1, min_child_weight=0, min_split_gain=0.0,
        n_estimators=num_trees, n_jobs=-1, num_leaves=31, objective="binary", 
        random_state=1, reg_alpha=0.0, reg_lambda=0.0, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=0,  )
m1.fit(X, y, verbose=1, eval_set = [(X,y)])

res_1 = m1.predict(X)

accuracy_score(res_1, y)

[1]	training's binary_logloss: 0.364431
[2]	training's binary_logloss: 0.30797
[3]	training's binary_logloss: 0.258656
[4]	training's binary_logloss: 0.239413
[5]	training's binary_logloss: 0.213851
[6]	training's binary_logloss: 0.204589
[7]	training's binary_logloss: 0.199272
[8]	training's binary_logloss: 0.193588
[9]	training's binary_logloss: 0.18745
[10]	training's binary_logloss: 0.172793
[11]	training's binary_logloss: 0.154085
[12]	training's binary_logloss: 0.150774
[13]	training's binary_logloss: 0.130422
[14]	training's binary_logloss: 0.118352
[15]	training's binary_logloss: 0.108654
[16]	training's binary_logloss: 0.09286
[17]	training's binary_logloss: 0.0890692
[18]	training's binary_logloss: 0.0813644
[19]	training's binary_logloss: 0.0745611
[20]	training's binary_logloss: 0.0607006
[21]	training's binary_logloss: 0.0568307
[22]	training's binary_logloss: 0.0529939
[23]	training's binary_logloss: 0.0461759
[24]	training's binary_logloss: 0.0445376
[25]	training's bina

0.9875141884222475

In [90]:
# log likelihood loss
def log_loss_train(labels, preds): #preds here is not y_pred, it's the sum_linear_pred (raw pred, a.k.a log odd)
    preds = 1. / (1. + np.exp(-preds)) # sigmoid convert the regression pred to probability (i.e., [0,1] range)
    grad = preds - labels
    hess = preds * (1. - preds)
    return grad, hess


def log_loss_valid(labels, preds):
    preds = 1. / (1. + np.exp(-preds))
    return '_log_loss_',log_loss(labels, preds),True



m_ = lightgbm.LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        importance_type='split', learning_rate=1, max_depth=max_depth_tree,
        min_child_samples=1, min_child_weight=0, min_split_gain=0.0,
        n_estimators=num_trees, n_jobs=-1, num_leaves=31, objective=log_loss_train, metrics = ["binary", 'binary_error'],
        random_state=1, reg_alpha=0.0, reg_lambda=0.0, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

# fitting model 
m_.fit(X, y, verbose=1, eval_set = [(X,y)], eval_metric=log_loss_valid)

res_ = m_.predict(X)

accuracy_score(res_, y)

[1]	training's binary_error: 0.169126	training's binary_logloss: 2.6015	training's _log_loss_: 0.369754
[2]	training's binary_error: 0.139614	training's binary_logloss: 1.76144	training's _log_loss_: 0.316638
[3]	training's binary_error: 0.103292	training's binary_logloss: 2.14285	training's _log_loss_: 0.282993
[4]	training's binary_error: 0.0885358	training's binary_logloss: 1.26876	training's _log_loss_: 0.245081
[5]	training's binary_error: 0.0851305	training's binary_logloss: 1.03233	training's _log_loss_: 0.228174
[6]	training's binary_error: 0.0737798	training's binary_logloss: 0.673541	training's _log_loss_: 0.198358
[7]	training's binary_error: 0.0658343	training's binary_logloss: 0.370674	training's _log_loss_: 0.182951
[8]	training's binary_error: 0.0703746	training's binary_logloss: 0.328558	training's _log_loss_: 0.212569
[9]	training's binary_error: 0.0590238	training's binary_logloss: 0.104922	training's _log_loss_: 0.173461
[10]	training's binary_error: 0.0556186	traini

[90]	training's binary_error: 0.0124858	training's binary_logloss: -2.93096	training's _log_loss_: 0.0251699
[91]	training's binary_error: 0.0124858	training's binary_logloss: -3.00721	training's _log_loss_: 0.0251174
[92]	training's binary_error: 0.0124858	training's binary_logloss: -2.98174	training's _log_loss_: 0.0251026
[93]	training's binary_error: 0.0124858	training's binary_logloss: -2.99002	training's _log_loss_: 0.0250987
[94]	training's binary_error: 0.0124858	training's binary_logloss: -2.99274	training's _log_loss_: 0.0250973
[95]	training's binary_error: 0.0124858	training's binary_logloss: -3.00135	training's _log_loss_: 0.0250843
[96]	training's binary_error: 0.0124858	training's binary_logloss: -3.00689	training's _log_loss_: 0.0250815
[97]	training's binary_error: 0.0124858	training's binary_logloss: -3.01168	training's _log_loss_: 0.0250563
[98]	training's binary_error: 0.0124858	training's binary_logloss: -3.01389	training's _log_loss_: 0.0250503
[99]	training's bin

0.9875141884222475

In [35]:
m1

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        importance_type='split', learning_rate=1, max_depth=5,
        metrics=['log_loss'], min_child_samples=1, min_child_weight=0,
        min_split_gain=0.0, n_estimators=100, n_jobs=-1, num_leaves=31,
        objective=<function loglikelihood at 0x7f1beeb3aae8>,
        random_state=1, reg_alpha=0.0, reg_lambda=0.0, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [28]:
y_pred

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [15]:
max_depth_tree =5
num_trees=1000

m2 = lightgbm.LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        importance_type='split', learning_rate=1, max_depth=max_depth_tree,
        min_child_samples=1, min_child_weight=0, min_split_gain=0.0,
        n_estimators=num_trees, n_jobs=-1, num_leaves=31, objective="binary", 
        random_state=1, reg_alpha=0.0, reg_lambda=0.0, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=0)
m2.fit(X, y, verbose=1, eval_set = [(X,y)])

res_2 = m2.predict(X)

accuracy_score(res_2, y)

[1]	training's binary_logloss: 0.364431
[2]	training's binary_logloss: 0.30797
[3]	training's binary_logloss: 0.258656
[4]	training's binary_logloss: 0.239413
[5]	training's binary_logloss: 0.213851
[6]	training's binary_logloss: 0.204589
[7]	training's binary_logloss: 0.199272
[8]	training's binary_logloss: 0.193588
[9]	training's binary_logloss: 0.18745
[10]	training's binary_logloss: 0.172793
[11]	training's binary_logloss: 0.154085
[12]	training's binary_logloss: 0.150774
[13]	training's binary_logloss: 0.130422
[14]	training's binary_logloss: 0.118352
[15]	training's binary_logloss: 0.108654
[16]	training's binary_logloss: 0.09286
[17]	training's binary_logloss: 0.0890692
[18]	training's binary_logloss: 0.0813644
[19]	training's binary_logloss: 0.0745611
[20]	training's binary_logloss: 0.0607006
[21]	training's binary_logloss: 0.0568307
[22]	training's binary_logloss: 0.0529939
[23]	training's binary_logloss: 0.0461759
[24]	training's binary_logloss: 0.0445376
[25]	training's bina

[284]	training's binary_logloss: 0.0248092
[285]	training's binary_logloss: 0.0248092
[286]	training's binary_logloss: 0.0248092
[287]	training's binary_logloss: 0.0248092
[288]	training's binary_logloss: 0.0248092
[289]	training's binary_logloss: 0.0248092
[290]	training's binary_logloss: 0.0248092
[291]	training's binary_logloss: 0.0248092
[292]	training's binary_logloss: 0.0248092
[293]	training's binary_logloss: 0.0248092
[294]	training's binary_logloss: 0.0248092
[295]	training's binary_logloss: 0.0248092
[296]	training's binary_logloss: 0.0248092
[297]	training's binary_logloss: 0.377646
[298]	training's binary_logloss: 0.534462
[299]	training's binary_logloss: 0.614099
[300]	training's binary_logloss: 2.10571
[301]	training's binary_logloss: 0.823719
[302]	training's binary_logloss: 2.96362
[303]	training's binary_logloss: 3.06715
[304]	training's binary_logloss: 3.00024
[305]	training's binary_logloss: 4.35151
[306]	training's binary_logloss: 3.14084
[307]	training's binary_log

[666]	training's binary_logloss: 2.90601
[667]	training's binary_logloss: 2.8276
[668]	training's binary_logloss: 3.69009
[669]	training's binary_logloss: 2.8276
[670]	training's binary_logloss: 2.90601
[671]	training's binary_logloss: 3.41566
[672]	training's binary_logloss: 2.90601
[673]	training's binary_logloss: 2.98441
[674]	training's binary_logloss: 2.90601
[675]	training's binary_logloss: 3.45486
[676]	training's binary_logloss: 2.90601
[677]	training's binary_logloss: 2.8276
[678]	training's binary_logloss: 3.02362
[679]	training's binary_logloss: 3.45486
[680]	training's binary_logloss: 2.90601
[681]	training's binary_logloss: 2.8276
[682]	training's binary_logloss: 3.53327
[683]	training's binary_logloss: 2.98441
[684]	training's binary_logloss: 2.90601
[685]	training's binary_logloss: 2.8276
[686]	training's binary_logloss: 3.53327
[687]	training's binary_logloss: 2.8276
[688]	training's binary_logloss: 3.06282
[689]	training's binary_logloss: 3.41566
[690]	training's binar

0.8910329171396141

In [20]:
import lightgbm as lgb


lgb_train = lgb.Dataset(X, y, free_raw_data=False)
lgb_eval = lgb.Dataset(X, y, free_raw_data=False)


# self-defined eval metric
# f(preds: array, train_data: Dataset) -> name: string, eval_result: float, is_higher_better: bool
# binary error
def binary_error(preds, train_data):
    labels = train_data.get_label()
    return 'binary_errorrr', np.mean(labels != (preds > 0.5)), False



def accuracy(preds, train_data):
    labels = train_data.get_label()
    return 'accuracy', np.mean(labels == (preds > 0.5)), True



def log_loss_(preds, train_data):
    y_preds = 1. / (1. + np.exp(-preds))
    labels = train_data.get_label()
    return 'log_loss_', log_loss(labels, y_preds), True



# log likelihood loss
def loglikelihood(preds, train_data): #preds here is not y_pred, it's the sum_linear_pred (raw pred)
    labels = train_data.get_label()
    preds = 1. / (1. + np.exp(-preds))
    grad = preds - labels
    hess = preds * (1. - preds)
    return grad, hess



params = {}
params = {}
params['learning_rate'] = 1
params['boosting_type'] = 'gbdt'
params['objective'] = 'binary'
params['metric'] = 'binary'
params['sub_feature'] = 1
params['num_leaves'] = 31
params['min_data'] = 0
params['max_depth'] = max_depth_tree
params['reg_alpha'] = 0
params['reg_lambda'] = 0

gbm = lgb.train(params,
                lgb_train,
                num_boost_round=num_trees,
                fobj=loglikelihood,
                feval=lambda preds, train_data: [log_loss_(preds, train_data),
                                                 accuracy(preds, train_data)],
                valid_sets=lgb_train)

y_pred = gbm.predict(X, num_iteration=gbm.best_iteration)

print(1-np.mean(y != (y_pred > 0.5)))


temp = 1. / (1. + np.exp(-y_pred))
print(1-np.mean(y != (temp > 0.5)))

[1]	training's binary_logloss: 2.6015	training's log_loss_: 0.369754	training's accuracy: 0.830874
[2]	training's binary_logloss: 1.76144	training's log_loss_: 0.316638	training's accuracy: 0.860386
[3]	training's binary_logloss: 2.14285	training's log_loss_: 0.282993	training's accuracy: 0.896708
[4]	training's binary_logloss: 1.26876	training's log_loss_: 0.245081	training's accuracy: 0.911464
[5]	training's binary_logloss: 1.03233	training's log_loss_: 0.228174	training's accuracy: 0.914869
[6]	training's binary_logloss: 0.673541	training's log_loss_: 0.198358	training's accuracy: 0.92622
[7]	training's binary_logloss: 0.371114	training's log_loss_: 0.182951	training's accuracy: 0.934166
[8]	training's binary_logloss: 0.329367	training's log_loss_: 0.212572	training's accuracy: 0.929625
[9]	training's binary_logloss: 0.101863	training's log_loss_: 0.173463	training's accuracy: 0.940976
[10]	training's binary_logloss: -0.256271	training's log_loss_: 0.162633	training's accuracy: 0.94

[89]	training's binary_logloss: -0.792212	training's log_loss_: 7.3343	training's accuracy: 0.785471
[90]	training's binary_logloss: -0.792212	training's log_loss_: 7.3343	training's accuracy: 0.785471
[91]	training's binary_logloss: -0.792212	training's log_loss_: 7.3343	training's accuracy: 0.785471
[92]	training's binary_logloss: -0.792212	training's log_loss_: 7.3343	training's accuracy: 0.785471
[93]	training's binary_logloss: -0.792212	training's log_loss_: 7.3343	training's accuracy: 0.785471
[94]	training's binary_logloss: -0.792212	training's log_loss_: 7.3343	training's accuracy: 0.785471
[95]	training's binary_logloss: -0.792212	training's log_loss_: 7.3343	training's accuracy: 0.785471
[96]	training's binary_logloss: -0.792212	training's log_loss_: 7.3343	training's accuracy: 0.785471
[97]	training's binary_logloss: -0.792212	training's log_loss_: 7.3343	training's accuracy: 0.785471
[98]	training's binary_logloss: -0.792212	training's log_loss_: 7.3343	training's accuracy:

[173]	training's binary_logloss: -0.792212	training's log_loss_: 7.3343	training's accuracy: 0.785471
[174]	training's binary_logloss: -0.792212	training's log_loss_: 7.3343	training's accuracy: 0.785471
[175]	training's binary_logloss: -0.792212	training's log_loss_: 7.3343	training's accuracy: 0.785471
[176]	training's binary_logloss: -0.792212	training's log_loss_: 7.3343	training's accuracy: 0.785471
[177]	training's binary_logloss: -0.792212	training's log_loss_: 7.3343	training's accuracy: 0.785471
[178]	training's binary_logloss: -0.792212	training's log_loss_: 7.3343	training's accuracy: 0.785471
[179]	training's binary_logloss: -0.792212	training's log_loss_: 7.3343	training's accuracy: 0.785471
[180]	training's binary_logloss: -0.792212	training's log_loss_: 7.3343	training's accuracy: 0.785471
[181]	training's binary_logloss: -0.792212	training's log_loss_: 7.3343	training's accuracy: 0.785471
[182]	training's binary_logloss: -0.792212	training's log_loss_: 7.3343	training's

[281]	training's binary_logloss: -0.792212	training's log_loss_: 7.3343	training's accuracy: 0.785471
[282]	training's binary_logloss: -0.792212	training's log_loss_: 7.3343	training's accuracy: 0.785471
[283]	training's binary_logloss: -0.792212	training's log_loss_: 7.3343	training's accuracy: 0.785471
[284]	training's binary_logloss: -0.792212	training's log_loss_: 7.3343	training's accuracy: 0.785471
[285]	training's binary_logloss: -0.792212	training's log_loss_: 7.3343	training's accuracy: 0.785471
[286]	training's binary_logloss: -0.792212	training's log_loss_: 7.3343	training's accuracy: 0.785471
[287]	training's binary_logloss: -0.792212	training's log_loss_: 7.3343	training's accuracy: 0.785471
[288]	training's binary_logloss: -0.792212	training's log_loss_: 7.3343	training's accuracy: 0.785471
[289]	training's binary_logloss: -0.792212	training's log_loss_: 7.3343	training's accuracy: 0.785471
[290]	training's binary_logloss: -0.792212	training's log_loss_: 7.3343	training's

[375]	training's binary_logloss: -0.792212	training's log_loss_: 7.3343	training's accuracy: 0.785471
[376]	training's binary_logloss: -0.792212	training's log_loss_: 7.3343	training's accuracy: 0.785471
[377]	training's binary_logloss: -0.792212	training's log_loss_: 7.3343	training's accuracy: 0.785471
[378]	training's binary_logloss: -0.792212	training's log_loss_: 7.3343	training's accuracy: 0.785471
[379]	training's binary_logloss: -0.792212	training's log_loss_: 7.3343	training's accuracy: 0.785471
[380]	training's binary_logloss: -0.792212	training's log_loss_: 7.3343	training's accuracy: 0.785471
[381]	training's binary_logloss: -0.792212	training's log_loss_: 7.3343	training's accuracy: 0.785471
[382]	training's binary_logloss: -0.792212	training's log_loss_: 7.3343	training's accuracy: 0.785471
[383]	training's binary_logloss: -0.792212	training's log_loss_: 7.3343	training's accuracy: 0.785471
[384]	training's binary_logloss: -0.792212	training's log_loss_: 7.3343	training's

...

[592]	training's binary_logloss: -0.792212	training's log_loss_: 7.3343	training's accuracy: 0.785471
[593]	training's binary_logloss: -0.792212	training's log_loss_: 7.3343	training's accuracy: 0.785471
[594]	training's binary_logloss: -0.792212	training's log_loss_: 7.3343	training's accuracy: 0.785471
[595]	training's binary_logloss: -0.792212	training's log_loss_: 7.3343	training's accuracy: 0.785471
[596]	training's binary_logloss: -0.792212	training's log_loss_: 7.3343	training's accuracy: 0.785471
[597]	training's binary_logloss: -0.792212	training's log_loss_: 7.3343	training's accuracy: 0.785471
[598]	training's binary_logloss: -0.792212	training's log_loss_: 7.3343	training's accuracy: 0.785471
[599]	training's binary_logloss: -0.792212	training's log_loss_: 7.3343	training's accuracy: 0.785471
[600]	training's binary_logloss: -0.792212	training's log_loss_: 7.3343	training's accuracy: 0.785471
[601]	training's binary_logloss: -0.792212	training's log_loss_: 7.3343	training's

[682]	training's binary_logloss: -0.792212	training's log_loss_: 7.3343	training's accuracy: 0.785471
[683]	training's binary_logloss: -0.792212	training's log_loss_: 7.3343	training's accuracy: 0.785471
[684]	training's binary_logloss: -0.792212	training's log_loss_: 7.3343	training's accuracy: 0.785471
[685]	training's binary_logloss: -0.792212	training's log_loss_: 7.3343	training's accuracy: 0.785471
[686]	training's binary_logloss: -0.792212	training's log_loss_: 7.3343	training's accuracy: 0.785471
[687]	training's binary_logloss: -0.792212	training's log_loss_: 7.3343	training's accuracy: 0.785471
[688]	training's binary_logloss: -0.792212	training's log_loss_: 7.3343	training's accuracy: 0.785471
[689]	training's binary_logloss: -0.792212	training's log_loss_: 7.3343	training's accuracy: 0.785471
[690]	training's binary_logloss: -0.792212	training's log_loss_: 7.3343	training's accuracy: 0.785471
[691]	training's binary_logloss: -0.792212	training's log_loss_: 7.3343	training's

[781]	training's binary_logloss: -0.792212	training's log_loss_: 7.3343	training's accuracy: 0.785471
[782]	training's binary_logloss: -0.792212	training's log_loss_: 7.3343	training's accuracy: 0.785471
[783]	training's binary_logloss: -0.792212	training's log_loss_: 7.3343	training's accuracy: 0.785471
[784]	training's binary_logloss: -0.792212	training's log_loss_: 7.3343	training's accuracy: 0.785471
[785]	training's binary_logloss: -0.792212	training's log_loss_: 7.3343	training's accuracy: 0.785471
[786]	training's binary_logloss: -0.792212	training's log_loss_: 7.3343	training's accuracy: 0.785471
[787]	training's binary_logloss: -0.792212	training's log_loss_: 7.3343	training's accuracy: 0.785471
[788]	training's binary_logloss: -0.792212	training's log_loss_: 7.3343	training's accuracy: 0.785471
[789]	training's binary_logloss: -0.792212	training's log_loss_: 7.3343	training's accuracy: 0.785471
[790]	training's binary_logloss: -0.792212	training's log_loss_: 7.3343	training's

[871]	training's binary_logloss: -0.792212	training's log_loss_: 7.3343	training's accuracy: 0.785471
[872]	training's binary_logloss: -0.792212	training's log_loss_: 7.3343	training's accuracy: 0.785471
[873]	training's binary_logloss: -0.792212	training's log_loss_: 7.3343	training's accuracy: 0.785471
[874]	training's binary_logloss: -0.792212	training's log_loss_: 7.3343	training's accuracy: 0.785471
[875]	training's binary_logloss: -0.792212	training's log_loss_: 7.3343	training's accuracy: 0.785471
[876]	training's binary_logloss: -0.792212	training's log_loss_: 7.3343	training's accuracy: 0.785471
[877]	training's binary_logloss: -0.792212	training's log_loss_: 7.3343	training's accuracy: 0.785471
[878]	training's binary_logloss: -0.792212	training's log_loss_: 7.3343	training's accuracy: 0.785471
[879]	training's binary_logloss: -0.792212	training's log_loss_: 7.3343	training's accuracy: 0.785471
[880]	training's binary_logloss: -0.792212	training's log_loss_: 7.3343	training's

[962]	training's binary_logloss: -0.792212	training's log_loss_: 7.3343	training's accuracy: 0.785471
[963]	training's binary_logloss: -0.792212	training's log_loss_: 7.3343	training's accuracy: 0.785471
[964]	training's binary_logloss: -0.792212	training's log_loss_: 7.3343	training's accuracy: 0.785471
[965]	training's binary_logloss: -0.792212	training's log_loss_: 7.3343	training's accuracy: 0.785471
[966]	training's binary_logloss: -0.792212	training's log_loss_: 7.3343	training's accuracy: 0.785471
[967]	training's binary_logloss: -0.792212	training's log_loss_: 7.3343	training's accuracy: 0.785471
[968]	training's binary_logloss: -0.792212	training's log_loss_: 7.3343	training's accuracy: 0.785471
[969]	training's binary_logloss: -0.792212	training's log_loss_: 7.3343	training's accuracy: 0.785471
[970]	training's binary_logloss: -0.792212	training's log_loss_: 7.3343	training's accuracy: 0.785471
[971]	training's binary_logloss: -0.792212	training's log_loss_: 7.3343	training's

In [19]:
print(accuracy_score(res_2, y))
accuracy_score(res_2, y) == (1-np.mean(y != (y_pred > 0.5)))
(1-np.mean(y != (y_pred > 0.5)))

0.8910329171396141


0.547105561861521

In [ ]:
# lightgbm.create_tree_digraph(m2, tree_index=0, show_info=["split_gain", "internal_value", "internal_count", "leaf_count"] )

In [ ]:
# lightgbm.create_tree_digraph(gbm, tree_index=0, show_info=["split_gain", "internal_value", "internal_count", "leaf_count"] )

In [ ]:
def loglikelihood(preds, train_data):
    labels = train_data.get_label()
    preds = 1. / (1. + np.exp(-preds))
    grad = preds - labels
    hess = preds * (1. - preds)
    return grad, hess


# self-defined eval metric
# f(preds: array, train_data: Dataset) -> name: string, eval_result: float, is_higher_better: bool
# binary error
def binary_error(preds, train_data):
    labels = train_data.get_label()
    return 'error', np.mean(labels != (preds > 0.5)), False


print('Finished 40 - 50 rounds with self-defined objective function and eval metric...')


# another self-defined eval metric
# f(preds: array, train_data: Dataset) -> name: string, eval_result: float, is_higher_better: bool
# accuracy
def accuracy(preds, train_data):
    labels = train_data.get_label()
    return 'accuracy', np.mean(labels == (preds > 0.5)), True


gbm = lgb.train(params,
                lgb_train,
                num_boost_round=100,
                fobj=loglikelihood,
                feval=lambda preds, train_data: [binary_error(preds, train_data),
                                                 accuracy(preds, train_data)],
                valid_sets=lgb_eval)

In [ ]:
y_pred = gbm.predict(X, num_iteration=gbm.best_iteration) > 0.5
accuracy_score(y_pred, y)

objetive = binary   => regresson on (0,1) => 1.1 => sigmoid(1.1) (this is y_pred) => calc error => finish 1 round, return  y_pred=sigmoid(1.1) 

objetive = log_loss_customer_func   => regresson on (0,1) => 1.1 (this is y_pred) => sigmoid(1.1) => calc error => finish 1 round,  return  y_pred=1.1

hessian ?
how to find best split
how to minimize error after know index for best split